In [ ]:
!pip install langgraph langchain_core langchain_groq langchain_huggingface langchain_chroma

In [31]:
from typing import TypedDict,Literal,Annotated,List
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
import os
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
from google.colab import userdata
from langgraph.graph.message import add_messages

In [54]:
class GraphState(TypedDict):
  request:Annotated[List,add_messages]
  classification:Literal['sales','support','general','remote']

def sales_message(state:GraphState):
  return {'request':'Our Sales team will be reaching out you soon'}

def remote_message(state:GraphState):
  return {'request':'Our remote team will be reaching out you soon'}

def support_message(state:GraphState):
  return {'request':'Our Support team will be reaching out you soon'}

def general_message(state:GraphState):
  return {'request':'Concerned team will be reaching out you soon'}

def classify_message(state:GraphState):
  llm=ChatGroq(model='llama-3.1-8b-instant')

  response=llm.invoke(f"""Classifiy the below user request message in to any 1 of  below categories:('sales','support','remote','general')
                          user message :{state['request']}
                          Provide your in one word """)
  classification=response.content.strip().lower()
  print(classification)
  return {'classification':classification}
def router(state:GraphState):

  classify=state.get('classification')
  if classify=='sales':
    return 'sales'
  elif classify =='support':
    return 'support'
  elif classify=='remote':
    return 'remote'
  else:
    return 'general'


In [55]:
graph=StateGraph(GraphState)

graph.add_node('classification',classify_message)
graph.add_node('sales_team',sales_message)
graph.add_node('support_team',support_message)
graph.add_node('general_team',general_message)
graph.add_node('remote_team',remote_message)

graph.set_entry_point('classification')
graph.add_conditional_edges(
    'classification',
    router,
    {'sales':'sales_team',
     'support':'support_team',
     'general':'general_team',
     'remote':'remote_team'}
)
graph.add_edge('sales_team',END)
graph.add_edge('support_team',END)
graph.add_edge('general_team',END)
graph.add_edge('remote_team',END)

agent=graph.compile()

In [69]:
response=agent.invoke({'request':'How to reach out for buying Gaming lapto'})

sales


In [71]:
for req in response['request']:
  req.pretty_print()

================================ Human Message =================================

How to reach out for buying Gaming lapto
================================ Human Message =================================

Our Sales team will be reaching out you soon
